# RUN IN JULIA

In [1]:
# import Pkg
# Pkg.add("HDF5")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames
using HDF5


In [3]:
Tx = CSV.read("inputs/Tx.csv", DataFrame, drop=1:1) |> Matrix #N, N
Ty = CSV.read("inputs/Ty.csv",DataFrame, drop=1:1)|> Matrix; #M, N
Tz = CSV.read("inputs/Tz.csv",DataFrame, drop=1:1)|> Matrix; #N, M

M,N = size(Ty)
Cf = 1
Cw = 1
Ct = 1
alpha = 0.5
println("M: ", M)
println("N: ", N)

M: 90
N: 10


In [4]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 120)

@variable(model, o[1:M], Bin)
@variable(model, x[1:N, 1:N, 1:N], Bin)
@variable(model, y[1:N, 1:M, 1:N], Bin)
@variable(model, z[1:N, 1:N, 1:M], Bin)
@variable(model, f[1:N] >= 0)
@variable(model, L >= 0)

@objective(model, Min, alpha*(Cf*sum(o[i] for i in 1:M) + 
Ct * sum(y[k, i, j] for k in 1:N, i in 1:M, j in 1:N) + 
Cw * sum(sum(y[k,i,j]*Ty[i,j] + z[k,j,i]*Tz[j,i] for i in 1:M, j in 1:N) + sum(x[k, j1, j2] * Tx[j1, j2] for j1 in 1:N, j2 in 1:N) for k in 1:N)) +
(1 - alpha) * L)

# Optional additional constriant
for j in 1:N, k in 1:N
    @constraint(model, x[k,j,j] == 0)
end
# end additional constraint

for k in 1:N
    @constraint(model, sum(y[k,i,j] for i in 1:M, j in 1:N) <= 1)
end

for k in 1:N
    @constraint(model, sum(x[k,j1,j2] for j1 in 1:N, j2 in 1:N) <= (N - 1) * sum(y[k,i,j] for i in 1:M, j in 1:N))
end

for i in 1:M
    @constraint(model, sum(y[k,i,j] for k in 1:N, j in 1:N) <= N * o[i])
end

for k in 1:N, i in 1:M
    @constraint(model, sum(z[k,j,i] for j in 1:N) == sum(y[k,i,j] for j in 1:N))
end

for j2 in 1:N 
    @constraint(model, sum(sum(y[k,i,j2] for i in 1:M) + sum(x[k,j1,j2] for j1 in 1:N) for k in 1:N) == 1)
end 

for j1 in 1:N 
    @constraint(model, sum(sum(x[k,j1,j2] for j2 in 1:N) + sum(z[k,j1,i] for i in 1:M) for k in 1:N) == 1)
end 

# Now time constraints
for k in 1:N 
    @constraint(model, f[k] == sum(y[k,i,j] * Ty[i,j] for i in 1:M, j in 1:N) + sum(x[k,j1,j2] * Tx[j1,j2] for j1 in 1:N, j2 in 1:N))
end

for k in 1:N 
    @constraint(model, f[k] <= L)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-05
Set parameter TimeLimit to value 120


In [5]:
optimize!(model)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1150 rows, 19101 columns and 76120 nonzeros
Model fingerprint: 0x9575be04
Variable types: 11 continuous, 19090 integer (19090 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 70.5000000
Presolve removed 110 rows and 100 columns
Presolve time: 0.07s
Presolved: 1040 rows, 19001 columns, 66720 nonzeros
Variable types: 0 continuous, 19001 integer (19000 binary)

Root relaxation: objective 1.290000e+01, 171 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [6]:
rm("results/output.h5")
h5write("results/output.h5", "factories", value.(o))
h5write("results/output.h5", "x_edges", value.(x))
h5write("results/output.h5", "y_edges", value.(y))
h5write("results/output.h5", "z_edges", value.(z))
